# 03 - Extract per-frame embeddings (EfficientNet-B3 backbone)
This notebook produces embeddings/<split>/<video_stem>.npy for each video (shape: T x feat_dim).
It will use your local spatial checkpoint if you provide its path; otherwise it will use a timm pretrained EfficientNet-B3 backbone.
Run cell-by-cell. This is GPU-accelerated and resumable.
